# Comprendre et manipuler la tokenisation dans les LLM

Ce notebook a pour objectif d’introduire la tokenisation, une étape fondamentale
dans le fonctionnement des modèles de langage de grande taille (LLM).

La tokenisation constitue l’interface entre le langage naturel et les modèles
numériques : elle détermine comment un texte est découpé, représenté et interprété
par un modèle.

À la fin de ce notebook, vous serez capable de :
- expliquer ce qu’est un token et un tokenizer,
- comparer différentes stratégies de tokenisation,
- manipuler des tokenizers modernes utilisés dans les LLM,
- comprendre l’impact de la tokenisation sur les performances et les coûts.


____

# Pourquoi la tokenisation est nécessaire
## 1. Du texte au numérique

Un modèle de langage est un modèle mathématique : il ne peut pas traiter directement
du texte brut. Toute information doit être représentée sous forme numérique.

La tokenisation est le processus qui permet de transformer un texte en une séquence
d’unités discrètes appelées *tokens*, lesquelles seront ensuite converties en
représentations numériques exploitables par le modèle.


## 2. Qu’est-ce qu’un token ?

Un token est l’unité de base manipulée par un modèle de langage.
Il ne correspond pas nécessairement à un mot entier : il peut s’agir d’une partie de
mot, d’un caractère, d’un nombre ou d’un symbole.

Le découpage en tokens dépend entièrement du *tokenizer* utilisé.
Un même texte peut donc être découpé différemment selon le modèle.


## 3. Qu’est-ce qu’un tokenizer ?

Un tokenizer est le composant chargé de transformer un texte brut en une séquence de tokens selon un ensemble de règles et de paramètres appris ou définis. 

Il détermine la manière dont le texte est découpé, quels symboles sont reconnus comme des unités valides et comment ces unités sont associées à des identifiants numériques. 

Le tokenizer joue un rôle central dans le fonctionnement d’un modèle de langage, car il conditionne la représentation du texte, la taille du vocabulaire, la longueur des séquences et, par conséquent, les performances et le coût computationnel du modèle. 

Chaque tokenizer est spécifiquement conçu pour être compatible avec un modèle donné, et toute incohérence entre le tokenizer et le modèle entraîne une dégradation significative des résultats.

____

# Les différentes tokenisations

## Tokenisation naïve par mots

Le texte est découpé en mots entiers séparés par des espaces ou des signes de ponctuation : 

![](../images/tokenisation.png)

In [8]:
text = "Les modèles de langage sont fascinants."
tokens = text.split()
tokens

['Les', 'modèles', 'de', 'langage', 'sont', 'fascinants.']

Cette approche est simple mais pose rapidement problème :
- le vocabulaire devient très grand,
- les mots inconnus ne peuvent pas être représentés,
- la généralisation est limitée.

## Tokenisation par caractères

Chaque caractère du texte est traité comme un token, garantissant une couverture complète du vocabulaire au prix de séquences très longues : 

In [9]:
list(text)

['L',
 'e',
 's',
 ' ',
 'm',
 'o',
 'd',
 'è',
 'l',
 'e',
 's',
 ' ',
 'd',
 'e',
 ' ',
 'l',
 'a',
 'n',
 'g',
 'a',
 'g',
 'e',
 ' ',
 's',
 'o',
 'n',
 't',
 ' ',
 'f',
 'a',
 's',
 'c',
 'i',
 'n',
 'a',
 'n',
 't',
 's',
 '.']

La tokenisation par caractères résout le problème des mots inconnus,
mais elle produit des séquences très longues, ce qui augmente fortement
le coût computationnel.

## Tokenisation par sous-mots

Les LLM modernes utilisent majoritairement des tokenizations basées sur des sous-mots.
L’idée est de découper les mots en unités fréquentes, réutilisables, permettant
de gérer efficacement les mots rares ou inconnus.

Exemple :
- "incompréhensible" → "in" + "compréhens" + "ible"


![](../images/tokenisation_illustration.jpg)

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = "Tokenization is computationally fascinating."
tokens = tokenizer.tokenize(text)
tokens


['token', '##ization', 'is', 'computational', '##ly', 'fascinating', '.']

On observe que le texte est découpé en sous-unités, et non en mots stricts. 

De plus, le préfixe `##` indique que le token est une continuation d’un mot précédent.



In [11]:
token_ids = tokenizer.encode(text)
token_ids

[101, 19204, 3989, 2003, 15078, 2135, 17160, 1012, 102]

Les modèles ne manipulent pas directement les tokens textuels, mais leurs identifiants numériques associés.


## Tokenisation au niveau des bytes (byte-level tokenization)  

Le texte est tokenisé à partir des octets qui le composent, assurant une représentation universelle et indépendante de la langue.

In [12]:
text = "Prix : 199,99€ 😄"

# Encodage UTF-8 en bytes
byte_tokens = text.encode("utf-8")

byte_tokens


b'Prix : 199,99\xe2\x82\xac \xf0\x9f\x98\x84'

In [13]:
list(byte_tokens)

[80,
 114,
 105,
 120,
 32,
 58,
 32,
 49,
 57,
 57,
 44,
 57,
 57,
 226,
 130,
 172,
 32,
 240,
 159,
 152,
 132]

In [14]:
bytes(byte_tokens).decode("utf-8")

'Prix : 199,99€ 😄'

## Tokenisation hybride 

Plusieurs niveaux de tokenisation sont combinés afin d’équilibrer robustesse, efficacité et précision selon le contexte d’utilisation.

Principe :
* on tokenise d’abord par mots connus,
* sinon on découpe en sous-mots,
* sinon on retombe au niveau byte ou caractère.

In [15]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "ChatGPTification ultra-méta-quantique 😄"
tokens = tokenizer.tokenize(text)

tokens

['Chat',
 'G',
 'PT',
 'ification',
 'Ġultra',
 '-',
 'm',
 'Ã©',
 'ta',
 '-',
 'quant',
 'ique',
 'ĠðŁĺ',
 'Ħ']

In [16]:
tokenizer.encode(text)

[30820,
 38,
 11571,
 2649,
 14764,
 12,
 76,
 2634,
 8326,
 12,
 40972,
 2350,
 30325,
 226]

## Impact pratique de la tokenisation

Les limites de contexte des LLM sont exprimées en nombre de tokens,
pas en nombre de mots ou de caractères.

Deux textes de longueur similaire peuvent donc avoir des coûts très différents
selon leur tokenisation.

Les modèles OpenAI utilisent un tokenizer spécifique optimisé pour leurs modèles.
La bibliothèque `tiktoken` permet de reproduire exactement la tokenisation
utilisée en production.

Cela permet notamment :
- de compter précisément le nombre de tokens,
- d'estimer le coût d'une requête API,
- de vérifier la compatibilité avec la limite de contexte.


In [24]:
import tiktoken

# Charger le tokenizer associé au modèle
tokenizer = tiktoken.encoding_for_model("gpt-4o")

# Exemple de texte à tokeniser
sample_text = "Every moment is a beginning"

# Tokeniser le texte
token_ids = tokenizer.encode(sample_text)

# Afficher les résultats
print("Text:", sample_text)
print("Token ids:", token_ids)
print("Number of tokens:", len(token_ids))

# Décoder les tokens pour vérifier la correspondance avec le texte original
decoded_text = tokenizer.decode(token_ids)
print("Decoded text:", decoded_text)

print("Nombre de mots:", len(sample_text.split()))
print("Nombre de caractères:", len(sample_text))

# -------------------------------------------------------
# Estimation du coût (exemple pédagogique concret)
# -------------------------------------------------------

# Supposons un tarif fictif mais réaliste :
# 5 $ par million de tokens en entrée
# 15 $ par million de tokens en sortie

price_input_per_million = 5.0
price_output_per_million = 15.0

input_tokens = len(token_ids)
output_tokens = 50  # Supposons que le modèle génère 50 tokens en réponse

estimated_cost = (
    (input_tokens / 1_000_000) * price_input_per_million
    + (output_tokens / 1_000_000) * price_output_per_million
)

print(f"Estimation du coût (exemple fictif) : ${estimated_cost:.8f}")


Text: Every moment is a beginning
Token ids: [15745, 4205, 382, 261, 10526]
Number of tokens: 5
Decoded text: Every moment is a beginning
Nombre de mots: 5
Nombre de caractères: 27
Estimation du coût (exemple fictif) : $0.00077500


Ici : 
- Le texte est converti en une séquence d'identifiants numériques.
- Chaque identifiant correspond à un token du vocabulaire du modèle.
- Le nombre de tokens n'est pas nécessairement égal au nombre de mots.

Les API de modèles de langage facturent généralement au nombre de tokens.
Nous illustrons ici une estimation de coût en utilisant un exemple tarifaire fictif
(les prix réels dépendent du modèle et peuvent évoluer).

____

# Comparaison de tokenizers

Tous les modèles de langage ne tokenisent pas le texte de la même manière.
Chaque modèle est entraîné avec un tokenizer spécifique, basé sur des choix
techniques différents (WordPiece, Byte-Level BPE, SentencePiece, etc.).

Nous allons comparer trois tokenizers associés à des modèles populaires :

- BERT (WordPiece)
- GPT-2 (Byte-Level BPE)
- XLM-RoBERTa (SentencePiece)

L'objectif est d'observer :
- les différences de découpage,
- le nombre de tokens générés,
- l'impact des choix de tokenisation.

In [19]:
models = [
    "bert-base-uncased",
    "gpt2",
    "xlm-roberta-base"
]

text = "L'internationalisation des modèles de langage est complexe."

for model in models:
    tok = AutoTokenizer.from_pretrained(model)
    print(f"\nTokenizer: {model}")
    print(tok.tokenize(text))
    print("Nombre de tokens:", len(tok.tokenize(text)))



Tokenizer: bert-base-uncased
['l', "'", 'international', '##isation', 'des', 'model', '##es', 'de', 'lang', '##age', 'est', 'complex', '##e', '.']
Nombre de tokens: 14

Tokenizer: gpt2
['L', "'", 'international', 'isation', 'Ġdes', 'Ġmod', 'Ã¨', 'les', 'Ġde', 'Ġlang', 'age', 'Ġest', 'Ġcomplex', 'e', '.']
Nombre de tokens: 15

Tokenizer: xlm-roberta-base
['▁L', "'", 'international', 'isation', '▁des', '▁modèles', '▁de', '▁lang', 'age', '▁est', '▁complexe', '.']
Nombre de tokens: 12


## 1. BERT (WordPiece)

- Les sous-mots sont indiqués par le préfixe `##`.
- Exemple : "internationalisation" → "international" + "##isation".
- Les accents sont supprimés (modèles uncased).
- Le texte français est partiellement dégradé ("modèles" → "model" + "##es").

BERT est optimisé pour l'anglais, ce qui explique certaines approximations
sur le français.

Nombre de tokens : 14


## 2. GPT-2 (Byte-Level BPE)

- Les espaces sont encodés avec le symbole `Ġ`.
- Les caractères accentués sont décomposés en bytes ("modèles" → "Ã¨").
- Le modèle fonctionne au niveau des bytes, ce qui garantit une couverture universelle.

GPT-2 n'a pas été spécifiquement entraîné pour le français,
ce qui explique certains découpages peu naturels.

Nombre de tokens : 15


## 3. XLM-RoBERTa (SentencePiece)

- Le symbole `▁` représente un espace.
- Les mots français sont mieux conservés ("modèles", "complexe").
- Le modèle est multilingue, ce qui améliore la tokenisation du français.

Nombre de tokens : 12


## Cas limites

In [20]:
texts = [
    "ChatGPTification ultra-méta-quantique",
    "Prix : 199,99€ 😄",
    "anticonstitutionnellement"
]

for t in texts:
    print(t)
    print(tokenizer.tokenize(t))
    print()


ChatGPTification ultra-méta-quantique
['Chat', 'G', 'PT', 'ification', 'Ġultra', '-', 'm', 'Ã©', 'ta', '-', 'quant', 'ique']

Prix : 199,99€ 😄
['P', 'rix', 'Ġ:', 'Ġ199', ',', '99', 'âĤ¬', 'ĠðŁĺ', 'Ħ']

anticonstitutionnellement
['ant', 'icon', 'st', 'itution', 'nel', 'lement']



Ces exemples illustrent plusieurs propriétés importantes du tokenizer GPT-2 :

1. Les mots inconnus ou néologismes sont découpés en sous-unités connues.
2. Les caractères accentués, symboles et emojis sont représentés via un encodage byte-level.
3. Les mots longs sont segmentés en fragments morphologiques fréquents.

Cela montre que la tokenisation moderne vise un compromis entre :
- robustesse universelle (via les bytes),
- efficacité du vocabulaire,
- capacité de généralisation.


____

# Conclusion

La tokenisation est une étape fondamentale dans les LLM.
Elle conditionne la représentation du texte, la longueur des séquences,
les performances des modèles et leur coût d’utilisation.

Comprendre la tokenisation, c’est comprendre comment un modèle de langage
lit réellement le langage humain.
